# The tale of the snake and the travelling salesman

## Abstract
In this paper we will discuss the mechanics behind the game of Snake and its application in graph theory.

# Objectives
Research and expand upon the following
1. Discuss what is the snake game.
2. How is the snake game related to graph theory?
3. What is a graph?
4. What is the travelling salesman problem?
5. What is a Hamiltonian path and cycle?
6. Implement and discuss an algorithm for generating a Hamiltonian cycle.
7. Implement and discuss an algorithm for the snake to follow the path to the food.

# Bonus objectives (if there's time)
1. Explore the rendering pipeline and how to visualize the snake
2. Research the compute shader step of the pipeline to compute the algorithms in steps 6. and 7.

# Snake The Game
![Snake2](images/snake-day-FEATURED-desktop.png)
## Overview
Snake is a mobile game where the player controls a lengthening line, called the snake. The gameplay consists of rectangular grid of blocks, a snake and a food. The basic rules are the following:
1. The initial state of the game begins with a snake of length 1 and a food at two location on the board.
2. The snake can move in any of four directions: up, down, left, right. It can't move diagonally.
3. Once the snake eat's a food it's length increments by one and a new food spawns in unoccupied square on the board.
4. The snake wins, if it can collect all the foods on the board.
5. It loses, if its head collides with its body or any of the edges of the board.

## History
Snake is a mobile game designed for the monochrome 6110 Nokia phones$^{[1]}$. It was inspired by games like the arcade Blockage (1976), Bigfoot Bonkers (1977), Worm (1978) and Nibler (1982). The developer behind the game, Taneli Armano, was born in 1965 and had a passion for board games and computer programming. After studying computer science and and mathematics, he joined Nokia's user interface developing group and worked on the development of its first handsets. Taneli's interest in ringtones led him to programme the tones in some of the handsets, creating memorable musical accompaniments. Snake has become a staple in the mobile gaming industry, with many users recalling the tunes it provided$^{[2]}$.

The development of the game was faced with some limitations. Among them were the number of keys to control the game, a limited phone display of 48x84 black and white pixels and small memory. The game was designed in the C programming language and was hand-written line-by-line without the use of specific tools or code generators. During testing, Taneli noticed that making a 90-degree turn without crashing was dificult, leading him to add a small delay to the crash to help players react and save the snake. This delay allowed players to continue playing even with faster speeds.

Snake has left an incredible legacy, with over 420 Snake-like games available on iOs alone. Nokia, which had 51% of the global market share in 2007, was baffled by the introduced demand from the iPhone for less battery life and fragile screens, which led to Nokia's decline and in 2014, Microsoft aquired its mobile phone busines.

# Graph Theory
A graph is a mathematical structure that consists of a set of objects in which some of the objects are "related" to each other in some sense$^{[3]}$. The object is often denoted by the term vertex (also refered to as a node or point), while the "relation" is refered to as an edge. A more formal definition is the following

Definition 1. A graph is a pair $G = (V, E)$, where $V$ is a set whose elements are called vertices, and $E$ is a set of unordered pairs $\{v_{1}, v_{2}\}$ of vertices, whose elements are called edges.

A graph can categorized in several ways, including direction and weight of the edges.

Definition 2. An undirected graph is such that 
$$\documentclass{article} 
\usepackage{tikz} 
\begin{document} 
\begin{tikzpicture}[main/.style = {draw, circle}] 
\node[main] (1) {$x_1$}; 
\end{tikzpicture} 
\end{document}$$

Graph Theory is the study of graphs.

# Bibliography
1. [The history of Snake: How the Nokia game defined a new era for the mobile industry, Ayla Angelos, Kyle Berger, 2021](https://www.itsnicethat.com/features/taneli-armanto-the-history-of-snake-design-legacies-230221)
2. [Nokia 1101 (1100) Ringtones, randomchannel1 TM, 2016](https://www.youtube.com/watch?v=vN88QnHfA7A)
3. [A Gentle Introduction to Graph Theory, Vaidehi Joshi, 2017](https://medium.com/basecs/a-gentle-introduction-to-graph-theory-77969829ead8)
4. 